Berikut adalah script untuk mengubah pdf lampiran profil kesehatan Jawa Barat jadi csv
Caveat: script setiap tahun berbeda karena halaman dan kerangka tabel berbeda-beda setiap tahunnya. Sehingga tidak dapat dilakukan automasi 
Feel free to give me any suggestions

In [19]:
# import library
import tabula
import camelot
import pandas as pd
import datetime as dt
import sys
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Append Data

In [2]:
def process_table(file_path, pages, year):
    df = tabula.read_pdf(file_path, pages=pages)[0]
    df.rename(columns={'Jenis Tenaga Kesehatan': "Jenis Tenaga", "Jumlah Tenaga Kesehatan": "Jumlah Tenaga"}, inplace=True)

    df['Jumlah Tenaga'] = pd.to_numeric(df['Jumlah Tenaga'].astype(str).str.replace("[^0-9]", "", regex=True), errors='coerce')

    df.dropna(subset=['Jumlah Tenaga'], inplace=True)
    df['Jumlah Tenaga'] = df['Jumlah Tenaga'].astype(int)
    
    df['tahun'] = year
    df['unit_kerja'] = "puskesmas"
    return df

In [3]:
## Data Tenaga Puskesmas
df_2022 = process_table("2022.pdf", "56", "2022")
df_2021 = process_table("2021.pdf", "43", "2021")
df_2020 = process_table("2020.pdf", "40", "2020")

tables_2019 = camelot.read_pdf("2019.pdf", flavor='stream', pages='41')
df_2019 = tables_2019[1].df.iloc[4:,:2].iloc[:9]
df_2019.set_axis(["Jenis Tenaga", "Jumlah Tenaga"], axis=1, inplace=True)
df_2019['Jumlah Tenaga'] = df_2019['Jumlah Tenaga'].astype(str).str.replace(",", "").astype(int)
df_2019['tahun'] = "2019"
df_2019['unit_kerja'] = "puskesmas"

data_2018 = {
    'Jenis Tenaga': ['Dokter atau dokter layanan primer', 'Dokter Gigi', 'Perawat', 'Bidan', 'Tenaga Kefarmasian',
                     'Tenaga Kesmas', 'Tenaga Kesehatan Lingkungan', 'Tenaga Gizi', 'Ahli Teknologi Laboratorium Medik'],
    'Jumlah Tenaga': [2097, 829, 9850, 13750, 889, 851, 674, 550, 514]
}
df_2018 = pd.DataFrame(data_2018)
df_2018['tahun'] = "2018"
df_2018['unit_kerja'] = "puskesmas"

final_df = pd.concat([df_2022, df_2021, df_2020, df_2019, df_2018], ignore_index=True)
final_df_pks = final_df.iloc[:,1:5]
final_df_pks.head()

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


,Jenis Tenaga,Jumlah Tenaga,tahun,unit_kerja
0,Dokter atau dokter layanan primer,2938,2022,puskesmas
1,Dokter gigi,9460,2022,puskesmas
2,Perawat,11605,2022,puskesmas
3,Bidan,17334,2022,puskesmas
4,Tenaga Kefarmasian,2133,2022,puskesmas


In [4]:
def process_data_tabula(file_path, pages, year, unit_kerja):
    # Membaca data dari file PDF menggunakan tabula
    df = tabula.read_pdf(file_path, pages=pages)[0]
    
    # Membersihkan data
    df.dropna(inplace=True)
    df.rename(columns={'Jenis Tenaga Kesehatan': "Jenis Tenaga", "Jumlah Tenaga Kesehatan": "Jumlah Tenaga"}, inplace=True)
    df['Jumlah Tenaga'] = pd.to_numeric(df['Jumlah Tenaga'].astype(str).str.replace(".", ""), errors='coerce').fillna(0).astype(int)
    
    # Menambahkan kolom tahun dan unit_kerja
    df['tahun'] = year
    df['unit_kerja'] = unit_kerja
    
    return df

def process_data_camelot(file_path, pages, year, unit_kerja):
    # Membaca data dari file PDF menggunakan camelot
    tables = camelot.read_pdf(file_path, flavor='stream', pages=pages)
    
    # Menggunakan df = tables[1].df untuk tahun 2019
    if year == "2019":
        df = tables[1].df
    else:
        df = tables[0].df
    
    # Membersihkan data
    df = df.iloc[2:,:2]
    df.set_axis(["Jenis Tenaga", "Jumlah Tenaga"], axis=1, inplace=True)
    df.rename(columns={'Jenis Tenaga': "Jenis Tenaga", "Jumlah Tenaga" : "Jumlah Tenaga"}, inplace=True)
    df['Jumlah Tenaga'] = pd.to_numeric(df['Jumlah Tenaga'].astype(str).str.replace(".", ""), errors='coerce').fillna(0).astype(int)
    
    # Menambahkan kolom tahun dan unit_kerja
    df['tahun'] = year
    df['unit_kerja'] = unit_kerja
    
    return df

In [5]:
## Jumlah Tenaga Rumah Sakit

df_2022 = process_data_tabula("2022.pdf", "60", "2022", "rumah sakit")
df_2021 = process_data_tabula("2021.pdf", "47", "2021", "rumah sakit")
df_2020 = process_data_camelot("2020.pdf", "44", "2020", "rumah sakit")
df_2019 = process_data_camelot("2019.pdf", "45", "2019", "rumah sakit")

data_2018 = {
    'Jenis Tenaga': ['Dokter atau dokter layanan primer', 'Dokter Gigi', 'Perawat', 'Bidan', 'Tenaga Kefarmasian', 'Tenaga Kesmas',
                     'Tenaga Kesehatan Lingkungan', 'Tenaga Gizi', 'Ahli Teknologi Laboratorium Medik'],
    'Jumlah Tenaga': [12002, 2436, 31024, 5831, 5229, 572, 225, 778, 4947]
}
df_2018 = pd.DataFrame(data_2018)
df_2018['tahun'] = "2018"
df_2018['unit_kerja'] = "rumah sakit"

final_df_rs = pd.concat([df_2022, df_2021, df_2020, df_2019, df_2018], ignore_index=True)

df_akum = pd.concat([final_df_rs, final_df_pks], ignore_index = True)
df_akum.to_csv("dataset/dataset_gbg/jumlah_jnstenaga.csv", index=False)
df_akum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Jenis Tenaga   85 non-null     object
 1   Jumlah Tenaga  85 non-null     int64 
 2   tahun          85 non-null     object
 3   unit_kerja     85 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.8+ KB


In [6]:
def read_and_clean_camelot(file_path, pages, year):
    tables = camelot.read_pdf(file_path, flavor='stream', pages=pages)
    df = tables[0].df
    df.columns = df.iloc[1]
    df = df.iloc[2:]
    df.reset_index(drop=True, inplace=True)
    df = df.rename(columns={"Tenaga Kesehatan": "jenis_tenaga"})  # Perbarui di sini
    df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].astype(float)
    df['tahun'] = year
    return df

In [7]:
### Ratio Dokter Gigi / 100.000 penduduk Jabar

df = tabula.read_pdf("2022.pdf", pages="61")[1]
df = df.iloc[3:,1:3]
df.set_axis(["jenis_tenaga","Ratio/100.000 Penduduk"], axis=1, inplace=True)
df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].str.replace(',','.')
df['Ratio/100.000 Penduduk'] = df['Ratio/100.000 Penduduk'].astype(float)
df.replace("Tenaga\rTeknisKefarmasian", "Tenaga Teknis Kefarmasian", inplace=True)

df2 = tabula.read_pdf("2022.pdf", pages="62")[0]
header_value = list(df2.columns)
df2 = df2.append(pd.Series(header_value, index=df2.columns), ignore_index=True)
df2 = df2.iloc[:,1:3]
df2.set_axis(["jenis_tenaga","Ratio/100.000 Penduduk"], axis=1, inplace=True)
df2['Ratio/100.000 Penduduk'] = df2['Ratio/100.000 Penduduk'].str.replace(',','.')
df2['Ratio/100.000 Penduduk'] = df2['Ratio/100.000 Penduduk'].astype(float)

df_2022 = pd.concat([df,df2], ignore_index = True)
df_2022['tahun'] = "2022"


df_2021 = read_and_clean_camelot("2021.pdf", "48", "2021")
df_2020 = read_and_clean_camelot("2020.pdf", "45", "2020")
df_2019 = tabula.read_pdf("2019.pdf", pages="46")[0]
df_2019.set_axis(["jenis_tenaga", "Ratio/100.000 Penduduk"], axis=1, inplace=True)
df_2019['tahun'] = "2019"

# Data 2018 (manual entry)
data_2018 = {
    'jenis_tenaga': ['Dokter Spesialis', 'Dokter Umum', 'Dokter Gigi', 'Bidan', 'Perawat', 'Kefarmasian',
                     'Kesehatan Masyarakat', 'Kesehatan Lingkungan', 'Gizi', 'Keterafian Fisik',
                     'Keteknisan Medis', 'Non Kesehatan'],
    'Ratio/100.000 Penduduk': [17.36, 12.12, 7.1, 40.22, 81.1, 13.41, 3.17, 1.98, 2.7, 1.68, 6.6, 30.2]
}
df_2018 = pd.DataFrame(data_2018)
df_2018['tahun'] = "2018"

# Menggabungkan DataFrames
df_rasio = pd.concat([df_2022, df_2021, df_2020, df_2019, df_2018], ignore_index=True)

df_rasio.to_csv("dataset/dataset_gbg/rasio_tenaga.csv", index=False)
df_rasio.head()

,jenis_tenaga,Ratio/100.000 Penduduk,tahun
0,Dokter Spesialis,22.31,2022
1,Dokter Umum,21.43,2022
2,Dokter Gigi,4.07,2022
3,Perawat,120.24,2022
4,Bidan,52.20,2022


In [8]:
## Data Kepemilikan Klinik Dokter Gigi Perorangan
data_2022 = {
    'Kabupaten/Kota': ['KABUPATEN BOGOR', 'KABUPATEN SUKABUMI', 'KABUPATEN CIANJUR', 'KABUPATEN BANDUNG', 'KABUPATEN GARUT',
                       'KABUPATEN TASIKMALAYA', 'KABUPATEN CIAMIS', 'KABUPATEN KUNINGAN', 'KABUPATEN CIREBON',
                       'KABUPATEN MAJALENGKA', 'KABUPATEN SUMEDANG', 'KABUPATEN INDRAMAYU', 'KABUPATEN SUBANG',
                       'KABUPATEN PURWAKARTA', 'KABUPATEN KARAWANG', 'KABUPATEN BEKASI', 'KABUPATEN BANDUNG BARAT',
                       'KABUPATEN PANGANDARAN', 'KOTA BOGOR', 'KOTA SUKABUMI', 'KOTA BANDUNG', 'KOTA CIREBON',
                       'KOTA BEKASI', 'KOTA DEPOK', 'KOTA CIMAHI', 'KOTA TASIKMALAYA', 'KOTA BANJAR'],
    'Jumlah': [298, 31, 0, 13, 6, 10, 8, 63, 251, 51, 27, 77, 13, 0, 276, 66, 0, 3, 75, 40, 0, 45, 66, 131, 69, 27, 16]
}
df_2022 = pd.DataFrame(data_2022)
df_2022['tahun'] = "2022"

tables_2021 = camelot.read_pdf("2021.pdf", flavor='stream', pages='158')
df_2021 = tables_2021[0].df
df_2021.columns = df_2021.iloc[3]
df_2021 = df_2021.iloc[9:, [1, 32]]
df_2021 = df_2021.iloc[:-1, :]
df_2021.set_axis(["Kabupaten/Kota", "Jumlah"], axis=1, inplace=True)
df_2021.reset_index(drop=True, inplace=True)
df_2021['Jumlah'] = df_2021['Jumlah'].str.replace("-", "0")
df_2021['Jumlah'] = df_2021['Jumlah'].replace('', '0')
df_2021['Jumlah'] = df_2021['Jumlah'].str.replace(",", "")
df_2021['Jumlah'] = pd.to_numeric(df_2021['Jumlah'], errors='coerce').fillna(0).astype(int)
df_2021['tahun'] = "2021"

tables_2020 = camelot.read_pdf("2020.pdf", flavor='stream', pages='149')
df_2020 = tables_2020[0].df
df_2020 = df_2020.iloc[10:, [1, 30]]
df_2020 = df_2020.iloc[:-1, :]
df_2020.set_axis(["Kabupaten/Kota", "Jumlah"], axis=1, inplace=True)
df_2020.reset_index(drop=True, inplace=True)
df_2020['Jumlah'] = df_2020['Jumlah'].str.replace("-", "0")
df_2020['Jumlah'] = df_2020['Jumlah'].replace('', '0')
df_2020['Jumlah'] = df_2020['Jumlah'].str.replace(".", "")
df_2020['Jumlah'] = pd.to_numeric(df_2020['Jumlah'], errors='coerce').fillna(0).astype(int)
df_2020['tahun'] = "2020"

tables_2019 = camelot.read_pdf("2019.pdf", flavor='stream', pages='139')
df_2019 = tables_2019[0].df
df_2019 = df_2019.iloc[11:, [1, 27]]
df_2019.set_axis(["Kabupaten/Kota", "Jumlah"], axis=1, inplace=True)
df_2019.reset_index(drop=True, inplace=True)
df_2019['Jumlah'] = df_2019['Jumlah'].str.replace("-", "0")
df_2019['Jumlah'] = df_2019['Jumlah'].replace('', '0')
df_2019['Jumlah'] = df_2019['Jumlah'].str.replace(".", "")
df_2019['Jumlah'] = pd.to_numeric(df_2019['Jumlah'], errors='coerce').fillna(0).astype(int)
df_2019['tahun'] = "2019"

df_klinik_org = pd.concat([df_2022, df_2021, df_2020, df_2019], ignore_index=True)
df_klinik_org.to_csv("dataset/dataset_gbg/data_klinik_indv.csv", index=False)
df_klinik_org.head()

,Kabupaten/Kota,Jumlah,tahun
0,KABUPATEN BOGOR,298,2022
1,KABUPATEN SUKABUMI,31,2022
2,KABUPATEN CIANJUR,0,2022
3,KABUPATEN BANDUNG,13,2022
4,KABUPATEN GARUT,6,2022


In [14]:
### PELAYANAN KESEHATAN GIGI DAN MULUT MENURUT KECAMATAN DAN PUSKESMAS

tables_2022 = camelot.read_pdf("2022.pdf", flavor='stream', pages='288')
df_2022 = tables_2022[0].df
df_2022.columns = df_2022.iloc[5]
df_2022 = df_2022.iloc[6:, [1, 3, 5, 9, 11, 13, 15]]
df_2022.reset_index(drop=True, inplace=True)
df_2022.iloc[27, 0] = "Jawa Barat"
df_2022.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan",
                  "Jumlah Kasus Gigi", "Jumlah Kasus dirujuk", "% Rasio Dirujuk"], axis=1, inplace=True)
df_2022 = df_2022.replace('', '0')
df_2022 = df_2022.replace('-', '0')
for col in ['Rasio Pencabutan', '% Rasio Dirujuk']:
    df_2022[col] = df_2022[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap', 'Jumlah Kasus Gigi', 'Jumlah Kasus dirujuk']:
    df_2022[col] = df_2022[col].str.replace('.', '').astype(int)
df_2022['tahun'] = "2022"
df_2022['unit_kerja'] = "puskesmas"

tables_2021 = camelot.read_pdf("2021.pdf", flavor='stream', pages='227')
df_2021 = tables_2021[0].df
df_2021.columns = df_2021.iloc[6]
df_2021 = df_2021.iloc[7:, [1, 3, 4, 6, 8, 10, 12]]
df_2021.reset_index(drop=True, inplace=True)
df_2021.iloc[27, 0] = "Jawa Barat"
df_2021.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan",
                  "Jumlah Kasus Gigi", "Jumlah Kasus dirujuk", "% Rasio Dirujuk"], axis=1, inplace=True)
df_2021 = df_2021.replace('', '0')
df_2021 = df_2021.replace('-', '0')
for col in ['Rasio Pencabutan', '% Rasio Dirujuk']:
    df_2021[col] = df_2021[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap', 'Jumlah Kasus Gigi', 'Jumlah Kasus dirujuk']:
    df_2021[col] = df_2021[col].str.replace(',', '').astype(int)
df_2021['tahun'] = "2021"
df_2021['unit_kerja'] = "puskesmas"

tables_2020 = camelot.read_pdf("2020.pdf", flavor='stream', pages='229')
df_2020 = tables_2020[0].df
df_2020 = df_2020.iloc[6:, [1, 3, 5, 7] + list(range(9, df_2020.shape[1]))]
df_2020.reset_index(drop=True, inplace=True)
df_2020.iloc[27, 0] = "Jawa Barat"
df_2020.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan",
                  "Jumlah Kasus Gigi", "Jumlah Kasus dirujuk", "% Rasio Dirujuk"], axis=1, inplace=True)
# Fill value Kota Banjar
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kota Banjar', 'Jumlah Tumpatan Gigi Tetap'] = "260"
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kota Banjar', 'Pencabutan Gigi Tetap'] = "881"
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kota Banjar', 'Rasio Pencabutan'] = "0.3"
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kota Banjar', 'Jumlah Kasus Gigi'] = "10872"
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kota Banjar', 'Jumlah Kasus dirujuk'] = "534"
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kota Banjar', '% Rasio Dirujuk'] = "0.049"
# Fill value Kota Tasik
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kota Tasikmalaya', 'Jumlah Kasus Gigi'] = "2225"

# Fill value Kab Ciamis
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kabupaten Ciamis', 'Jumlah Tumpatan Gigi Tetap'] = "559"
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kabupaten Ciamis', 'Pencabutan Gigi Tetap'] = "402"
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Kabupaten Ciamis', 'Rasio Pencabutan'] = "1.5"

df_2020 = df_2020.replace('', '0')
df_2020 = df_2020.replace('-', '0')
df_2020 = df_2020.replace('#DIV/0!', '0')
for col in ['Rasio Pencabutan', '% Rasio Dirujuk']:
    df_2020[col] = df_2020[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap', 'Jumlah Kasus Gigi', 'Jumlah Kasus dirujuk']:
    df_2020[col] = df_2020[col].str.replace('.', '').astype(int)
df_2020['tahun'] = "2020"
df_2020['unit_kerja'] = "puskesmas"

jawabarat_row = df_2020[df_2020['Kabupaten/Kota'] != 'Jawa Barat']
total_tumpatan = jawabarat_row['Jumlah Tumpatan Gigi Tetap'].sum()
total_pencabutan = jawabarat_row['Pencabutan Gigi Tetap'].sum()
total_kasus_gigi = jawabarat_row['Jumlah Kasus Gigi'].sum()
total_kasus_dirujuk = jawabarat_row['Jumlah Kasus dirujuk'].sum()

# Ganti nilai pada baris dengan Kabupaten/Kota == 'Jawa Barat'
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Jawa Barat', 'Jumlah Tumpatan Gigi Tetap'] = total_tumpatan
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Jawa Barat', 'Pencabutan Gigi Tetap'] = total_pencabutan
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Jawa Barat', 'Jumlah Kasus Gigi'] = total_kasus_gigi
df_2020.loc[df_2020['Kabupaten/Kota'] == 'Jawa Barat', 'Jumlah Kasus dirujuk'] = total_kasus_dirujuk


tables_2019 = camelot.read_pdf("2019.pdf", flavor='stream', pages='203')
df_2019 = tables_2019[0].df
df_2019 = df_2019.iloc[6:, [1, 3] + list(range(5, df_2019.shape[1]))]
df_2019.reset_index(drop=True, inplace=True)
df_2019.iloc[22, 4] = "163839"
df_2019.iloc[22, 6] = "0,04"
df_2019.iloc[27, 0] = "Jawa Barat"
df_2019.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan",
                  "Jumlah Kasus Gigi", "Jumlah Kasus dirujuk", "% Rasio Dirujuk"], axis=1, inplace=True)
df_2019 = df_2019.replace('', '0')
df_2019 = df_2019.replace('-', '0')
df_2019 = df_2019.replace('#DIV/0!', '0')
for col in ['Rasio Pencabutan', '% Rasio Dirujuk']:
    df_2019[col] = df_2019[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap', 'Jumlah Kasus Gigi', 'Jumlah Kasus dirujuk']:
    df_2019[col] = df_2019[col].str.replace('.', '').astype(int)
df_2019['tahun'] = "2019"
df_2019['unit_kerja'] = "puskesmas"

tables_2018 = camelot.read_pdf("2018.pdf", flavor='stream', pages='178')
df_2018 = tables_2018[0].df
df_2018 = df_2018.iloc[5:, 1:]
df_2018.reset_index(drop=True, inplace=True)
df_2018.loc[len(df_2018)] = ["Jawa Barat", "226974", "161703", "0.71"]
df_2018.set_axis(["Kabupaten/Kota", "Jumlah Tumpatan Gigi Tetap", "Pencabutan Gigi Tetap", "Rasio Pencabutan"],
                 axis=1, inplace=True)
df_2018 = df_2018.replace('', '0')
df_2018 = df_2018.replace('-', '0')
df_2018 = df_2018.replace('#DIV/0!', '0')
for col in ['Rasio Pencabutan']:
    df_2018[col] = df_2018[col].str.replace(',', '.').astype(float)

for col in ['Jumlah Tumpatan Gigi Tetap', 'Pencabutan Gigi Tetap']:
    df_2018[col] = df_2018[col].str.replace('.', '').astype(int)
df_2018['tahun'] = "2018"
df_2018['unit_kerja'] = "puskesmas"

df_gabung = pd.concat([df_2022, df_2021, df_2020, df_2019, df_2018], ignore_index=True)

df_gabung.to_csv("dataset/dataset_gbg/data_kasus_gigi.csv", index=False)
df_gabung.head()

,Kabupaten/Kota,Jumlah Tumpatan Gigi Tetap,Pencabutan Gigi Tetap,Rasio Pencabutan,Jumlah Kasus Gigi,Jumlah Kasus dirujuk,% Rasio Dirujuk,tahun,unit_kerja
0,Kabupaten Bogor,10905,11277,0.97,161427.0,5016.0,3.11,2022,puskesmas
1,Kabupaten Sukabumi,4712,6911,0.68,61684.0,2723.0,4.41,2022,puskesmas
2,Kabupaten Cianjur,2022,3577,0.57,27559.0,1269.0,4.60,2022,puskesmas
3,Kabupaten Bandung,19783,20738,0.95,210294.0,9846.0,4.68,2022,puskesmas
4,Kabupaten Garut,4241,7070,0.60,50119.0,889.0,1.77,2022,puskesmas


In [10]:
## Gabungan data jumlah dokter gigi per unit kerja

df_2018 = pd.read_csv("dataset/dg_unit_kerja/2018.csv")
df_2019 = pd.read_csv("dataset/dg_unit_kerja/2019.csv")
df_2020 = pd.read_csv("dataset/dg_unit_kerja/2020.csv")
df_2021 = pd.read_csv("dataset/dg_unit_kerja/2021.csv")
df_2022 = pd.read_csv("dataset/dg_unit_kerja/2022.csv")
df_gabung = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], ignore_index=True)
df_gabung.to_csv("dataset/dataset_gbg/jmlh_dg_unit_kerja.csv", index=False)
df_gabung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Kab/Kota     1156 non-null   object
 1   jumlah       1156 non-null   int64 
 2   jns_kelamin  1156 non-null   object
 3   profesi      1156 non-null   object
 4   unit_kerja   1156 non-null   object
 5   tahun        1156 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 54.3+ KB


In [18]:
## Gabung Jumlah Kasus gigi Anak SD Per Kab/Kota Puskesmaas

df_2018 = pd.read_csv("dataset/pelayanan_gigi_anaksd/2018.csv")
df_2019 = pd.read_csv("dataset/pelayanan_gigi_anaksd/2019.csv")
df_2020 = pd.read_csv("dataset/pelayanan_gigi_anaksd/2020.csv")
df_2021 = pd.read_csv("dataset/pelayanan_gigi_anaksd/2021.csv")
df_2022 = pd.read_csv("dataset/pelayanan_gigi_anaksd/2022.csv")
df_gabung = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], ignore_index=True)
df_gabung.to_csv("dataset/dataset_gbg/jmlh_kasusgg_sd.csv", index=False)
df_gabung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Kabupaten/Kota                    140 non-null    object 
 1   jumlah_SD/MI                      140 non-null    int64  
 2   jumlah_SD/MI_SG_Massal            140 non-null    int64  
 3   %jumlah_SD/MI_SG_Massal           140 non-null    float64
 4   jumlah_SD/MI_LG                   140 non-null    int64  
 5   %jumlah_SD/MI_LG                  140 non-null    float64
 6   jumlah_murid_SD/MI_L              140 non-null    int64  
 7   jumlah_murid_SD/MI_P              140 non-null    int64  
 8   jumlah_murid_SD/MI_LP             140 non-null    int64  
 9   jumlah_murid_SD/MI_L_Diperiksa    140 non-null    int64  
 10  %jumlah_murid_SD/MI_L_Diperiksa   140 non-null    float64
 11  jumlah_murid_SD/MI_P_Diperiksa    140 non-null    int64  
 12  %jumlah_